## Import investigator package of the PyOTIC software

In [ ]:
# Import os to easily join names of filepaths
import os

# Add the path of the PyOTIC Software to the system path
# Adjust this path to where the PyOTIC Software package is located
import sys
sys.path.append('../../')

#Load investigator package
import pyoti
pyoti.info()

#Create an experiment and open a new temporary experiment dbfile
experiment = pyoti.open_experiment()

## Create experiment file (or open previously saved one)

In [ ]:
# Choose the path the data should be loaded from
#
# datadir: The path where the data is located
# datafile: The name of the file that contains the data.
#datadir = '/srv/files/home/tobiasj/data/ASWAD/unwrapping/Henning/2016_05_04-Parameter_Controls/'
#datafile = 'test.bin'

In [ ]:
# save the state of the experiment in the database file
experiment.save(pack=True)

In [ ]:
# show status of Records, Views, MultiRegions, and Modifications in experiment
experiment.print_status()

In [ ]:
# revert changes since last commit of experiment
experiment.abort()

In [ ]:
# cleanup/pack database file
experiment.cleanup()

In [ ]:
# close database file
experiment.close()

## Create record(s) and add to experiment

In [ ]:
# Define a name for the record (defaults to 'default')
name='alpha'

# Define a function that is used to read in the data you want to analyze.
# The function needs to receive at least the positional parameter 'filename'.
# The return value of the function needs to be the data as a numpy array.
# You can (beside other options) use functions that the package numpy offers to read in data:
# http://docs.scipy.org/doc/numpy/reference/routines.io.html
#
# One example, to read in data from a text file with 5 header lines followed by the data,
# could look like this:

import numpy as np
import pyoti.data.labview as lv
import os

def load_data(filename):
    #data = np.loadtxt(filename, skiprows=5)
    #data = lv.read_labview_bin_data(filename)[:,0:3]
    return np.array([np.arange(30)]).T

# Define the samplingrate (either provide a function or simply a variable).
# The function gets executed once, upon initialisation of the record. The
# return value of the function (or the value of the variable) gets stored in
# the record object:
#def samplingrate():
#    samplingrate = 40000.0
#    return samplingrate
#samplingrate = 40000.0

# Name the traces here, the load_data() function returns. Make sure the 
# traces are properly describing the data returned by load_data function.
# This definition takes precedence over the traces defined in the
# configfile (see below)
traces = 'psdX'

# You can provide a configfile, which, for instance, defines the traces returned by load_data().
# If not provided, configfile defaults to '../pyoti/etc/GenericDataFile.cfg'.
# You could also create your own setup specific configfile and use GenericDataFile as a template.
# Make sure to also add the parameter cfgfile to the function call below, if you define a cfgfile,
# like: experiment.create_record(cfgfile=cfgfile, ...)
#cfgfile = '../pyoti/etc/record/GenericDataFile.cfg' 

record = experiment.create_record(name=name, traces=traces, load_data=load_data, filename='test.bin')

## Analyse and modify data

In [ ]:
name = 'one'
group = 'selection'
parent = 'alpha'

experiment.add_group(name, parent, group_type=group, adjust=False)
experiment.get_view(name).start = 0
experiment.get_view(name).stop = 10
experiment.adjust_view(name)

In [ ]:
name = 'two'
group = 'selection'
parent = 'alpha'

experiment.add_group(name, parent, group_type=group, adjust=False)
experiment.get_view(name).start = 10
experiment.get_view(name).stop = 20
experiment.adjust_view(name)

In [ ]:
name = 'three'
group = 'selection'
parent = 'alpha'

experiment.add_group(name, parent, group_type=group, adjust=False)
experiment.get_view(name).start = 20
experiment.get_view(name).stop = 30
experiment.adjust_view(name)

In [ ]:
experiment.add_view('concat', 'one', adjust=False)
experiment.append_to('concat', 'two')
experiment.append_to('concat', 'three')
experiment.adjust_view('concat')

In [ ]:
print(experiment.views.concat.sticky_start)
print(experiment.views.concat.sticky_stop)
experiment.views.concat.start = 5
experiment.views.concat.stop = 26
print(experiment.views.concat.sticky_start)
print(experiment.views.concat.sticky_stop)

In [ ]:
experiment.adjust_view('concat')